In [1]:
import pandas as pd

# Получение данных
file_name = 'case 729.csv'  # Откуда брать файл
data_source = pd.read_csv(file_name, sep=',', index_col=0,
                          skiprows=[1]  # Пропуск единиц измерения
                          ).dropna(
    axis=1, how='all')  # Чтение файла

# pd.options.display.float_format = '{:,.2f}'.format # Если нужно ограничить вывод до 2 знаков после запятой

# Разбиение таблицы на параметры и критерии
params = data_source.iloc[:, 0:6].astype(float)
dimEther_MCMF_column = data_source.iloc[:, 6].astype(float)
methanol_MCMF_column = data_source.iloc[:, 7].astype(float)
q_summ_column = data_source.iloc[:, 8].astype(float)


In [2]:
# Линейная интерполяция
from scipy.interpolate import LinearNDInterpolator
import numpy as np

# Создание точек для интерполяции
points = np.array(params.values)

# Создание интерполяторов
print('Расчет интерполяторов...')
print('Расчет интерполятора 1...')
dimEther_MCMF = LinearNDInterpolator(points, dimEther_MCMF_column.values)
print('Расчет интерполятора 2...')
methanol_MCMF = LinearNDInterpolator(points, methanol_MCMF_column.values)
print('Расчет интерполятора 3...')
q_summ = LinearNDInterpolator(points, q_summ_column.values)
print('Расчет интерполяторов завершен')

PRINT_PROGRESS = False  # Вывод прогресса
# Верхние и нижние границы параметров
X_LOWER = np.array([240, 340, 90, 1.3, 60, 110])
X_UPPER = np.array([260, 360, 110, 2, 80, 130])


Расчет интерполяторов...
Расчет интерполятора 1...
Расчет интерполятора 2...
Расчет интерполятора 3...
Расчет интерполяторов завершен


In [3]:
# Однокритериальная оптимизация
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса
from pymoo.core.problem import Problem
from pymoo.optimize import minimize


class Best_c1 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = -dimEther_MCMF(x)


class Best_c2 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = methanol_MCMF(x)


class Best_c3 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = q_summ(x)


res_c1 = minimize(Best_c1(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
res_c2 = minimize(Best_c2(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
res_c3 = minimize(Best_c3(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
print('Однокритериальная оптимизация для каждого критерия (Метод Хука-Дживс)')
print('Первый критерий')
print('X: ', res_c1.X)
print('F: ', res_c1.F)

print('Второй критерий')
print('X: ', res_c2.X)
print('F: ', res_c2.F)

print('Третий критерий')
print('X: ', res_c3.X)
print('F: ', res_c3.F)

# Сохранение результатов для однокритериальной оптимизации на основе маргинальных критериев
best_c1 = -res_c1.F
best_c2 = res_c2.F
best_c3 = res_c3.F


Однокритериальная оптимизация для каждого критерия (Метод Хука-Дживс)
Первый критерий
X:  [260. 360. 100.   2.  80. 130.]
F:  [-129.989587]
Второй критерий
X:  [260. 360.  90.   2.  80. 130.]
F:  [0.034958]
Третий критерий
X:  [260.  360.  100.    1.3  80.  130. ]
F:  [45009106.14443998]


In [4]:
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса


def objective_function(x):
    '''Маргинальная целевая функция'''
    сurrent_c1 = dimEther_MCMF(x)
    сurrent_c2 = methanol_MCMF(x)
    сurrent_c3 = q_summ(x)

    diff_c1 = best_c1 - сurrent_c1
    diff_c2 = сurrent_c2 - best_c2
    diff_c3 = сurrent_c3 - best_c3

    return np.sqrt(diff_c1**2 + diff_c2**2 + diff_c3**2)


# Постановка задачи однокритериальной оптимизации
class Single_objective_optimization(Problem):

    def __init__(self) -> None:
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = objective_function(x)


single_objective_problem = Single_objective_optimization()

# Запуск оптимизации
try:
    # Хука - Дживс
    res_PS = minimize(
        single_objective_problem, algorithm=PatternSearch(), seed=1)
    print('PS')
    print('X: ', res_PS.X)
    print('F: ', res_PS.F)
    print('C1: ', dimEther_MCMF(res_PS.X))
    print('C2: ', methanol_MCMF(res_PS.X))
    print('C3: ', q_summ(res_PS.X))

except Exception as e:
    print('Ошибка PS')
    print(e)


PS
X:  [260.  360.  100.    1.3  80.  130. ]
F:  [0.77518151]
C1:  [129.235012]
C2:  [0.212505]
C3:  [45009106.14444084]


In [ ]:
# Многокритериальная оптимизация
from pymoo.optimize import minimize
from pymoo.core.problem import Problem

# Алгоритмы оптимизации
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.age import AGEMOEA

# Постановка задачи


class Multi_objective_optimization(Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=3, n_constr=0,
                         # Минимальные значения параметров
                         xl=X_LOWER,
                         # Максимальные значения параметров
                         xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = -1 * dimEther_MCMF(x)
        f2 = methanol_MCMF(x)
        f3 = q_summ(x)
        out["F"] = np.column_stack([f1, f2, f3])


# Запуск оптимизации
problem = Multi_objective_optimization()

try:
    print('Многокритериальная оптимизация (Метод NSGA2)')
    res_NSGA2 = minimize(problem, algorithm=NSGA2(), seed=1,
                         verbose=PRINT_PROGRESS, termination=('n_gen', 50))  # Вывод прогресса)
    # print('X: ', res_NSGA2.X)
    # print('F: ', res_NSGA2.F)

    # Сохранение результатов
    res_NSGA2_df = pd.DataFrame(res_NSGA2.X)
    res_NSGA2_df['dimEther_MCMF'] = -1 * res_NSGA2.F[:, 0]
    res_NSGA2_df['methanol_MCMF'] = res_NSGA2.F[:, 1]
    res_NSGA2_df['q_summ'] = res_NSGA2.F[:, 2]
    res_NSGA2_df.to_csv('MOO NSGA2 Results.csv', sep=',', index=False,
                        header=data_source.columns.values)

except Exception as e:
    print('Ошибка NSGA2')
    print(e)

try:
    print('Многокритериальная оптимизация (Метод AGEMOEA)')
    res_AGEMOEA = minimize(problem, algorithm=AGEMOEA(),
                           seed=1, verbose=PRINT_PROGRESS)

    # print('X: ', res_AGEMOEA.X)
    # print('F: ', res_AGEMOEA.F)

    # Сохранение результатов
    res_AGEMOEA_df = pd.DataFrame(res_AGEMOEA.X)
    res_AGEMOEA_df['dimEther_MCMF'] = -1 * res_AGEMOEA.F[:, 0]
    res_AGEMOEA_df['methanol_MCMF'] = res_AGEMOEA.F[:, 1]
    res_AGEMOEA_df['q_summ'] = res_AGEMOEA.F[:, 2]
    res_AGEMOEA_df.to_csv('MOO AGEMOEA Results.csv', sep=',', index=False,
                          header=data_source.columns.values)
except Exception as e:
    print('Ошибка AGEMOEA')
    print(e)
